In [1]:
# for plotting interactive graph
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

# import GLMCC
import sys
import pandas as pd
import numpy as np
sys.path.append('../')

from glmcc import GLMCC  # model
from glmcc import spiketime_relative  # useful functions

Loading BokehJS ...

In [2]:
class GLMCC_bokeh(GLMCC):
    """
    to visualize with Bokeh, override plot method of the GLMCC class
    """
    def plot(self, t_sp, title, width=500, height=250):
        p = figure(title=title, background_fill_color="#fafafa", plot_width=width, plot_height=height)
        cc, edges = np.histogram(t_sp, bins=self.m, range=(-self.w, self.w))
        at = np.exp(self.theta[:self.m])
        j_ij = np.exp(self.theta[-2] * self.func_f(s=self.xk) + self.theta[:self.m])
        j_ji = np.exp(self.theta[-1] * self.func_f(s=-self.xk) + self.theta[:self.m])
        p.quad(top=cc, bottom=0, left=edges[:-1], right=edges[1:], fill_color="navy", line_color="white", alpha=0.5)

        # connectivity undetermined: gray, positive j: magenta, negative j: cyan
        colors = {0: 'gray', 1: 'cyan', 2: 'magenta'}
        p.line(self.xk, j_ij, line_width=3.0, line_color=colors[(abs(self.theta[-2]) >= self.j_thresholds[0]) * ((self.theta[-2] > 0) + 1)], alpha=0.7)
        p.line(self.xk, j_ji, line_width=3.0, line_color=colors[(abs(self.theta[-1]) >= self.j_thresholds[1]) * ((self.theta[-1] > 0) + 1)], alpha=0.7)
        p.line(self.xk, at, line_width=3.0, line_color='lime', alpha=0.7)
        return p


def estimate_connectivity(df_sp, reference_id, target_id, delay_list):
    spiketime_ref = list(df_sp.query('neuron==@reference_id').spiketime.values)
    spiketime_tar = list(df_sp.query('neuron==@target_id').spiketime.values)

    t_sp = spiketime_relative(spiketime_ref=spiketime_ref, spiketime_tar=spiketime_tar)
    
    for i, delay in enumerate(delay_list):
        if i == 0:
            glm = GLMCC_bokeh(delay=delay)
            glm.fit(t_sp, verbose=False)
            max_idx = i
            
        else:
            glm_tmp = GLMCC_bokeh(delay=delay)
            glm_tmp.fit(t_sp, verbose=False)

            if glm_tmp.max_log_posterior > glm.max_log_posterior:
                glm = glm_tmp
                max_idx = i
    
    title = 'connectivity from {} to {} (delay: {})'.format(reference_id, target_id, delay_list[max_idx])
    p = glm.plot(t_sp, title=title)
    return glm, p

In [3]:
df = pd.read_csv('./sample_data.csv')  # spike train in sec scale
df.head()

,neuron,spiketime
0,1,20.95
1,1,93.55
2,1,107.40
3,1,116.45
4,1,218.45


In [4]:
neurons = set(df.neuron)
neurons

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [5]:
plots = []
pairs = [(1,2), (1,3), (1,4), (1,5)]

for pair in pairs:
    glm, p = estimate_connectivity(df_sp=df, reference_id=pair[0], target_id=pair[1], 
                                   delay_list=[1.0, 2.0, 3.0, 4.0, 5.0])
    plots.append(p)
    
grid = gridplot(plots, ncols=2)
show(grid)